## SETUP PROJECT AND PATH to src\

why: in jupyter notebook I can't use __file__ to get to the project dir, so I need to use Path-cwd

In [40]:
import sys
import importlib

In [ ]:

# -------------------------------------------------------------------
# VARIABLES
# -------------------------------------------------------------------

# Project root = one folder above the current notebook
try:
    PROJECT_ROOT = Path(__file__).resolve().parent.parent
except NameError:  # when running inside Jupyter
    PROJECT_ROOT = Path.cwd().resolve().parent


# Path to src folder
SRC_PATH = PROJECT_ROOT / "src"

# Add src to sys.path if not already there
if str(SRC_PATH) not in sys.path:
    sys.path.insert(0, str(SRC_PATH))

# -------------------------------------------------------------------
# IMPORT UTILS
# -------------------------------------------------------------------

import utils
importlib.reload(utils)  # so edits to utils.py are reflected without restarting kernel
print("utils.py imported from:", SRC_PATH)

utils.py imported from: C:\Users\Vaccari\Desktop\iCloudDrive\Desktop\ENRICO\05_LEARNING\University\ToU\Phases\02_Calibration_Phase\Applied_Machine_Learning\Regression\beyond-grades-ml-project\src


In [48]:
csv_path = utils.get_student_csv_path()
print(csv_path)

C:\Users\Vaccari\Desktop\iCloudDrive\Desktop\ENRICO\05_LEARNING\University\ToU\Phases\02_Calibration_Phase\Applied_Machine_Learning\Regression\beyond-grades-ml-project\data\raw\student_performance_data_2025.csv


In [ ]:
df = utils.load_student_csv(csv_path)
utils.save_dataset(df, '01_dataset_raw.xlsx')

✅ File saved at: C:\Users\Vaccari\Desktop\iCloudDrive\Desktop\ENRICO\05_LEARNING\University\ToU\Phases\02_Calibration_Phase\Applied_Machine_Learning\Regression\beyond-grades-ml-project\data\processed\01_dataset_raw.xlsx
